In [2]:
import re
from collections import Counter


In [4]:
def words(text): return re.findall(r'\w+', text.lower())
WORDS = Counter(words(open('/content/big.txt').read()))

In [7]:
def P(c, word, N=sum(WORDS.values())):
  "Probability of a `word` ."
  return P_word(c,N)*P_error(word,c)

def P_word(c, N=sum(WORDS.values())):
  return WORDS[c] / N if c in WORDS else 1e-6

def P_error(word,c):
  edit_distance = levenshtein_distance(word,c)

  if edit_distance == 0:
    return 1

  elif edit_distance == 1:
    return 0.1

  elif edit_distance == 2:
    return 0.01

  else:
    return 0

In [8]:
def correction(word):
  return max(candidates(word), key=lambda c: P(c,word))

def candidates(word):
  return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
  return set(w for w in words if w in WORDS)

In [11]:
def edits1(word):
  letters = 'abcdefghijklmnopqrstuvwxyz'
  splits = [(word[:i],word[i:]) for i in range(len(word)+1)]
  deletes = [L+R[1:] for L,R in splits if R]
  transposes = [L+R[1]+R[0]+R[2:] for L,R in splits if len(R)>1]
  replaces = [L+c+R[1:] for L,R in splits if R for c in letters]
  inserts = [L+c+R for L,R in splits for c in letters]
  return set(deletes + transposes + replaces + inserts)

def edits2(word):
  return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def levenshtein_distance(a, b):
  n,m = len(a), len(b)
  if n == 0:return m
  if m == 0:return n

  dp = [[0]*(m+1) for _ in range(n+1)]

  for i in range(n+1):
    dp[i][0]=i
  for j in range(m+1):
    dp[0][j]=j
  for i in range(1,n+1):
    for j in range(1,m+1):
      cost = 0 if a[i-1]==b[j-1]else 1
      dp[i][j]=min(dp[i-1][j]+1, dp[i][j-1]+1, dp[i-1][j-1]+cost)
  return dp[n][m]

In [13]:
correction('speling')

'spelling'

In [14]:
correction('fesjn')

'resin'